In [3]:
# d6623, d6897
# Generate a model that classifies card values, from the images created
# Uses images in suits/train and suits/test (but test is not added yet)
# Heavily "inspired" by catsvdogs model from keras

# Flow
# 1. Load all the data
# 2. Train on model

import numpy as np
import tensorflow as tf
import os
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

base_dir = '/Users/James/Projects/PycharmProjects/jerry/cards'
train_dir = os.path.join(base_dir, 'value/train')
val_dir = os.path.join(base_dir, 'value/val')

In [4]:
def value_to_label(val):
    # Cause its really weird the order goes backwards
    # 10, 11, 12, 13, 1, 2, 3, 4, 5, 6, 7, 8, 9
    labels = [1, 10, 11, 12, 13, 2, 3, 4, 5, 6, 7, 8, 9]
    return labels[val]

train_datagen = ImageDataGenerator(rescale=1./255,  # Just to add some more datapoints
                                   brightness_range=[0.2, 2],
                                   shear_range=3.0,
                                   zoom_range=[.95, 1.05],
                                   rotation_range=5,
                                   validation_split=0.2)

#val_datagen = ImageDataGenerator(rescale=1./255)
# Replaced since it can flow from same directory

batch_size = 5
print("Making training generator")
train_generator = train_datagen.flow_from_directory(train_dir,
                                        target_size=(150,150),
                                        batch_size=batch_size,
                                        color_mode='grayscale',
                                        class_mode='categorical',
                                        subset='training',
                                        shuffle=True)

print("Making validation generator")
val_generator = train_datagen.flow_from_directory(train_dir,
                                                target_size=(150, 150),
                                                batch_size=batch_size,
                                                color_mode='grayscale',
                                                class_mode='categorical',
                                                subset='validation')

Making training generator
Found 91 images belonging to 13 classes.
Making validation generator
Found 13 images belonging to 13 classes.


In [ ]:
# Load some example data
img = plt.imread(os.path.join(train_dir, "1/club_0.jpg"))
plt.imshow(img)

img = np.expand_dims(img, axis=0)
ex_datagen = train_datagen
ex_datagen.fit(img)

plt.figure(figsize=(16,16))
it = ex_datagen.flow(img)
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(it.next().reshape(288,352, 3))  # Lol that reshape was a total guess
plt.show()

In [ ]:
length = 104  # how far to cycle to see every image
print('\nSample of example input train image')
for inputs_batch, labels_batch in train_generator:
    plt.matshow(inputs_batch[0].reshape(150,150))
    plt.show()

    print('label of', value_to_label(labels_batch[0].argmax(axis=0)))
    print('Shape of', inputs_batch.shape)
    break

print('\nSample of example input validation image')
for inputs_batch, labels_batch in val_generator:
    print(inputs_batch[0].shape)
    plt.matshow(inputs_batch[0].reshape(150,150))
    plt.show()

    print('label of', value_to_label(labels_batch[0].argmax(axis=0)))
    print('Shape of', inputs_batch.shape)
    break

In [6]:
from keras import layers, optimizers
from keras import models

# Probably too complex for this situiation, but I don't care
model = models.Sequential()
model.add(layers.Conv2D(64, (3,3), activation='relu', input_shape=(150,150,1)))
model.add(layers.MaxPooling2D((3,3)))  # Make the view larger; find bigger patterns

model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())  # Dimension reduce
model.add(layers.Dropout(0.5))  # Get rid of conspiracies
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(13, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 64)      640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 12800)             0

In [ ]:
from keras.callbacks import Callback
class CustomCallback(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        if epoch % 50 == 0:
            print('Epoch', epoch)


from time import time
start = time()
history = model.fit(train_generator,
                  steps_per_epoch=train_generator.samples // batch_size,  # 10 * 10 = 100 from 104 samples
                  epochs=500,
                  validation_data=val_generator,
                  validation_steps=val_generator.samples // batch_size,
                  verbose=0,
                  callbacks=[CustomCallback()])  # total of 128*100=12800 samples used
print('Took',(time()-start).__round__(4),'seconds to train the network')

In [7]:
save = False
load = True
export = False

if save:
    # save the model
    print('Saving model...')
    model.save('models/value-v4.h5')
    print('Saved.')

if load:
    # Load it
    print('Loading model...')
    model = models.load_model(os.path.join(base_dir, 'models/value-v4.h5'))

if export:
    print('Exporting model...')
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()

    with open('models/value-v4.tflite', 'wb') as f:
        f.write(tflite_model)

Loading model...


In [ ]:
# Visualise the training data
%matplotlib inline
import matplotlib.pyplot as plt

acc = history.history['acc']
loss = history.history['loss']
val_acc = history.history['val_acc']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training accuracy')
plt.legend()

plt.figure()  # Combines the two graphs

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training loss')
plt.legend()

plt.show()

In [11]:
# Test on custom image
from keras.preprocessing import image

print('Testing on all cards')

overall = [0,0]

for num in os.listdir(train_dir):
    if num != '.DS_Store':
        print('Testing num', num)

        dir = os.path.join(train_dir, num)

        fnames = []
        for fname in os.listdir(dir):
            if fname != '.DS_Store':
                fnames.append(os.path.join(dir, fname))

        overall[1] += fnames.__len__()
        correct = 0

        for img_path in fnames:
            img = image.load_img(img_path,
                                 target_size=(150,150),
                                 color_mode='grayscale')

            x = image.img_to_array(img)
            x = x.astype('float32')/255
            x = x.reshape((1,) + x.shape)  # Convert from (150,150,1) to (1,150,150,1)

            pred = model.predict(x, verbose=0)
            pred = pred.reshape(13)

            #print("\nPrediction of", value_to_label(pred.argmax(axis=0)))
            #print("With certainty of", pred[pred.argmax(axis=0)].__round__(2))

            if value_to_label(pred.argmax(axis=0)) == int(num):
                correct += 1
        print(correct, '/', fnames.__len__())
        overall[0] += correct
print('Overall accuracy of', (overall[0]/overall[1]*100).__round__(2),'%')
print('(%s/%s)' % (overall[0], overall[1]))

        #plt.imshow(img)
        #plt.show()




Testing on all cards
Testing num 9
8 / 8
Testing num 11
8 / 8
Testing num 7
8 / 8
Testing num 6
8 / 8
Testing num 1
8 / 8
Testing num 10
6 / 8
Testing num 8
8 / 8
Testing num 4
8 / 8
Testing num 3
8 / 8
Testing num 12
8 / 8
Testing num 2
8 / 8
Testing num 13
8 / 8
Testing num 5
8 / 8
Overall accuracy of 98.08 %
(102/104)
